In [ ]:
#finding average duration of each activity

import pandas as pd

df = pd.read_csv("newest_aruba.csv")
newdf=pd.DataFrame(df)

#newdf["Avg_Duration"]=""
newdf["MI"]=""

meandf=newdf.groupby(by="Annotation")["Duration"].mean()
meandf
newest_df=pd.DataFrame()
#newest_df.insert(loc=0, column='Avg_Duration', value=newdf.groupby(by="Annotation")["Duration"].mean())
#newest_df.insert(loc=1, column='Annotation', value=sorted(newdf['Annotation'].unique()))


#ann=sorted(newdf['Annotation'].unique())
#newestdf['Avg_Duration']=newdf.groupby(by="Annotation")["Duration"].mean()
#newdf['Avg_Duration']
#print(newest_df)
newdf.groupby("Annotation")["Duration"].mean()

newest_df.insert(loc=0, column='Avg_Duration', value=newdf.groupby(by="Annotation")["Duration"].mean())
#newest_df.insert(loc=1, column='Annotation', value=sorted(newdf['Annotation'].unique()))

print(newest_df)
#newest_df.to_csv("new_aruba.csv")

In [ ]:
## Frequency of sensors
#%%timeit
import pandas as pd
import numpy as np

df = pd.read_csv("newest_aruba.csv")
col=['M003','M002','M007','M004','M005','M018','M019','M015','M016','M020','M017','M009','M013','M031','M021','M012','M014','M008','M006','M030','M010','M026','M022','M028','M027','M029','M023','M025','M001','M024','M011','D001','D002']
newdf=df[col]
newdf.index=df.Annotation
print(newdf)
sum_df=newdf.sum(axis='columns',min_count=1)
#sum_df.index=newdf.Annotation
print(sum_df)
final_df=newdf.div(sum_df, axis=0).round(3)
print("Final :\n",final_df)
# final_df=final_df.groupby(by="Annotation")[col].mean()
# print(final_df)
deno=df['Annotation'].value_counts()
print(deno)
print("******")
newest_df=final_df.groupby(by="Annotation")[col].sum(min_count=1)
print(newest_df)
print("+++++++")
f_df=newest_df.div(deno, axis=0).round(2)
print(f_df)
f_df[col] = f_df[col].replace({0:np.nan})
print(f_df)
f_df=f_df.stack()
print(f_df)
f_df.to_csv("StackIFreq2.csv")

In [ ]:
## Preactivity

import pandas as pd
from collections import Counter

d=pd.read_csv("newest_aruba.csv")
df=pd.DataFrame(d)
ann=df['Annotation'].unique().tolist()
print(ann)


df2=pd.DataFrame()
df3=pd.DataFrame()
df4=pd.DataFrame()

# print(len(df))
# print(df)

for i in range(len(ann)):
    pre=[]
    for j in range(len(df)):
        if df.iloc[j].Annotation==ann[i]:
            #print(ann[i],df.iloc[j].Annotation)
            pre.append(df.iloc[j].preact)
    #print(pre)
    unic=Counter(pre)
    #print(unic)
    df2 = pd.DataFrame.from_records(list(dict(unic).items()), columns=['PreAct','Freq'])
    #print(df2)
    print(df2)
    summation=df2['Freq'].sum()
    print(summation)
    #df3=pd.DataFrame(df2['Freq'].div(summation,axis=0),columns=['Freq'])
    df3=df2['Freq'].div(summation,axis=0)
    #df3.columns=['Freq_P']
    print(df3)
    df2=pd.concat([df2,df3],axis=1)
    
    df2["Annotation"]=ann[i]
    print(df2)
    df4=pd.concat([df4,df2],axis=0)
    #df4.set_index([ann[i]])
    #df2 = df.append(pd.Series(pre, index = [ind]), ignore_index=True)
print(df4)
df4.to_csv('Train_Preact.csv')

In [ ]:
## Shift

import pandas as pd
from collections import Counter

d=pd.read_csv("newest_aruba.csv")
df=pd.DataFrame(d)
ann=df['Annotation'].unique().tolist()
print(ann)

df2=pd.DataFrame()
df3=pd.DataFrame()
df4=pd.DataFrame()

# print(len(df))
# print(df)

for i in range(len(ann)):
    pre=[]
    for j in range(len(df)):
        if df.iloc[j].Annotation==ann[i]:
            #print(ann[i],df.iloc[j].Annotation)
            pre.append(df.iloc[j].A_Shift)
    print("111")
    #print(pre)
    unic=Counter(pre)
    print(unic)
    df2 = pd.DataFrame.from_records(list(dict(unic).items()), columns=['Shift','Freq'])
    #print(df2)
    print(df2)
    summation=df2['Freq'].sum()
    print(summation)
    #df3=pd.DataFrame(df2['Freq'].div(summation,axis=0),columns=['Freq'])
    df3=df2['Freq'].div(summation,axis=0)
    #df3.columns=['Freq_P']
    print(df3)
    df2=pd.concat([df2,df3],axis=1)
    
    df2["Annotation"]=ann[i]
    print(df2)
    df4=pd.concat([df4,df2],axis=0)
    #df4.set_index([ann[i]])
    #df2 = df.append(pd.Series(pre, index = [ind]), ignore_index=True)
print(df4)
#df4.to_csv('Shift.csv')

In [ ]:
## SEGMENTATION

from owlready2 import *

import pandas as pd
from collections import Counter
import time

st=time.time()# get the start time

onto = get_ontology("dummy_ssn1.owl").load()

default_world.full_text_search_properties.append(comment)

data=pd.read_csv("Temp6.csv", usecols=['Time','Sensor'])
df=pd.DataFrame(data)

listt=['Relax','Sleeping','Wash_Dishes','Work','Eating','Respirate','Bed_to_Toilet','Housekeeping']

f_df=pd.DataFrame()
topK={}
preAct=""
def startSensor(sen,indx,p):
    start=onto.search(type = onto.FeatureOfInterest, hasStartSensor = [sen[0]])  #activity with n as the start sensor
    print("Activity of Start :",start)
    if indx!=0:
        if len(start) == 1:
            s_i=indx
            A_s_n=start[0]
            print("Start index : ", s_i)
            return s_i,A_s_n
        elif len(start)>1:
            if start[0]==onto.Work:
                s_n=start[0]
                print("++++onto.", s_n)
            else:
                s_n=checkPreAct(start,p)
                print("Start index : ", indx)
            return indx,s_n
        else:
            return -1,"null"
    return indx,start[0]

def getSubString(l_n,sym):
    pos=l_n.find(sym)
    
    if pos>=0:
        s_s=l_n[0:pos]
    else: 
        s_s=l_n
    print(s_s)
    return s_s

def preActivity(Act):
    i=0
    pre={}
    preAct={}
    p=""
    for s, o in onto.hasPreActivityProb.get_relations():
        if(i==57):
            break
        else:
            i=i+1
            print(i)
            print("Activity", s , "has Pre Acticity Probability : ", o )
            if s!=p:
                print("******")
                max=0.0
            if s in Act:
                print([s.name, float(o.name)])
                if float(o.name)>max:
                    max=float(o.name)
                    an=o
                pre[s]=an
                print("PreActivity: ",pre)
            p=s
            print(p)
    print(pre) 
    for key,value in pre.items():
        print(value.__class__)
        cl=value.__class__.name
        print("cllc ",cl)
        pos=cl.find('0')
        if pos>=0:
            s_s=cl[pos+1:]
        else: 
            s_s=cl
        print(s_s)
        preAct[key]=s_s
    return preAct
        
def checkPreAct(p1,p2):
    
    ditt=['Meal_Preparation','Eating','Leave_Home','Work','Housekeeping','Sleeping','Bed_to_Toilet' ]
    dit2={}
#     dit3={}
#     dit4={}
    
    dit=preActivity(p1)
#     ditt[onto.Leave_Home]='Meal_Preparation'
#     dit2[onto.Leave_Home]='Eating'
#     dit3[onto.Leave_Home]='Leave_Home' 
#     dit4[onto.Leave_Home]='Work'
#     dit4[onto.Leave_Home]='Housekeeping'
    dit2[onto.Meal_Preparation]='Leave_Home'
    for k,v in dit.items():
        if k.name=='Meal_Preparation':
            for x in listt:
                if x==p2 or dit2[k]==p2:
                    return k
        if k.name=='Leave_Home':
            for y in ditt:
                if y==p2:
                    return k
        if v==p2 :
            return k
    return p2
            
    

def findTopK():
    for s, o in onto.hasTopK.get_relations():
        print("Activity", s , "has Top K : ", o )
        print([s.name, o.name])
        topK[s.name]=int(o.name)
        print(topK)
    
def FreqCheck(Act):
    freq=[]
    i=0
    for s, o in onto.hasFreqProbOf.get_relations():
        if(i==90):
            break
        else:
            i=i+1
            print(i)
            print("Activity", s , "has Frequency Probability : ", o )
            if s==Act :
                print([s.name, float(o.name)])
                freq.append(o)
                print(freq)
    print(freq)
    j=0
    se=[]
    dic={}
    for a, b in onto.hasSensor.get_relations():
        if(j==86):
            break
        else:
            j=j+1
            print(j)
            print("Prob",a, "has Sensor : ", b)
            for i in freq[len(freq)-topK[Act.name]:] :
                if i==a:
                    print([float(a.name),b.name])
                    se.append(b)
                    print(se)
                    dic[float(a.name)]=b.name
                    print(dic)
    print(se)
    return dic
    
def findSensor(s,fr):
    for i in fr.values():
        if i==s:
            print("yes i: ",i,"has sensor: ",i)
            return 1
    print("No i: ",i,"has no sensor: ",i)
    return 0
 
def skipSensor(A,j):
    print("%%%%%%")
    s=df.iloc[j+1].Sensor
    print("s : ",s)
    sen=default_world.search(comment = FTS(s))   #finding the sensor individual using string annotation search
    print(sen)
    lo=onto.search(type = onto.Deployment, deployedSystem = [sen[0]])  # location of the sensor
    lo_name=lo[0].name   # getting only the location name as a string
    print(lo_name)
    sen_loc=getSubString(lo_name,'_')
    print("@@@@",sen_loc)
    
    if sen_loc=="Front" or sen_loc=="Back":
        return 1
    
    Actv=default_world.search(comment = FTS(sen_loc))   #searching for activity with location loc_name
    print("!!!!AL: ",Actv)
    Actv_name=Actv[0].name                    # getting only the activity name as string
    print(A)
    if A in Actv:
        print("YES")
        return 1
    else:
        return 0
    
def segment(start_ind,cut_ind,A_name,t_df):
    seg=df[start_ind:cut_ind]
    seg.columns=['Time',A_name]
    t_df=pd.concat([t_df,seg],axis=1)
    return t_df
    


flag=0
Activity_name=""
f_list=[]
i=0
start_index=0
findTopK()
pAct=""
skip_count=0
for i in range(len(df)):
    
    x=df.iloc[i].Sensor
    print("x : ",x)
    
    sensor=default_world.search(comment = FTS(x))   #finding the sensor individual using string annotation search
    print(sensor)
    if i==0:
        sensr=sensor
        ind=i
    #sensr=sensor
    if flag==0:
        print("/////////")
        start_index,Activity=startSensor(sensr,ind,pAct)  #getting the Activity if sensor is the start sensor
        print(Activity)
        if Activity=="null":
            sensr=sensor
            ind=i
            continue
        print("PreActivity: ",pAct," has Activity : ",Activity)
        freq_list=FreqCheck(Activity)
        Activity_name=Activity.name
#         else:
#             sensr=sensor
#             ind=i
#             continue
        
        
    loc=onto.search(type = onto.Deployment, deployedSystem = [sensor[0]])  # location of the sensor
    loc_name=loc[0].name   # getting only the location name as a string
    
    sensor_loc=getSubString(loc_name,'_')
    
    if sensor_loc=="Front" or sensor_loc=="Back":
        continue
    
    Act=default_world.search(comment = FTS(sensor_loc))   #searching for activity with location loc_name
    print("AL: ",Act)
    Act_name=Act[0].name                    # getting only the activity name as string
    
    sensor_present=findSensor(sensor[0].name,freq_list)
    
    if Activity_name != Act_name and sensor_present==0 and i+1!=len(df):
#         print("$$$$$")
#         if skip>5: 
            print("$$$$$")
            skip=skipSensor(Activity,i)
            if skip==1 and skip_count<=5:
                #sensr=
                skip_count=skip_count+1
                print("@#$%^&&",i)
                #continue
            else:
                cut_index=i
                f_df=segment(start_index,cut_index,Activity_name,f_df)
                sensr=sensor
                print("Sensor : ",sensr)
                ind=i
                print("SKipping sensor: ", ind)
                pAct=Activity_name
                print("Previous Avtivity: ", pAct)
                flag=0
                continue
        
    if i+1==len(df) :
        print("!!!!")
        cut_index=i+1
        f_df=segment(start_index,cut_index,Activity_name,f_df)
        
    else:
        print("@@@@@",i)
        flag=1
        
print(f_df[:40])
f_df.to_csv('Segment.csv')

# get the end time
et = time.time()

# get the execution time
elapsed_time = et - st
print('Execution time:', elapsed_time, 'seconds')